In [3]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

In [4]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.metrics import (precision_score, f1_score, recall_score, accuracy_score, 
average_precision_score ,auc,roc_auc_score)
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [47]:
df = pd.read_csv(r'C:\Users\Kumar\OneDrive - The City University of New York\Desktop\NYL Data Analysis\notebook\final_encoded_dataset.csv')

In [48]:
df.head(5)

,Unnamed: 0,loan_amnt,int_rate,installment,annual_inc,dti,open_acc,pub_rec,revol_bal,revol_util,...,application_type_JOINT,zip_column_05113,zip_column_11650,zip_column_22690,zip_column_29597,zip_column_30723,zip_column_48052,zip_column_70466,zip_column_86630,zip_column_93700
0,0,6400.0,15.61,223.78,60000.0,25.72,12.0,0.0,31228.0,97.6,...,0,0,0,0,0,0,1,0,0,0
1,1,25000.0,19.99,662.21,67000.0,20.49,18.0,1.0,24755.0,42.1,...,0,1,0,0,0,0,0,0,0,0
2,2,15000.0,5.32,451.73,59000.0,11.50,12.0,0.0,8925.0,23.3,...,0,1,0,0,0,0,0,0,0,0
3,3,16000.0,15.61,559.44,72000.0,29.40,8.0,0.0,6972.0,79.2,...,0,1,0,0,0,0,0,0,0,0
4,4,8725.0,12.12,290.30,50000.0,22.88,10.0,0.0,25397.0,45.8,...,0,0,0,0,0,0,0,0,0,0


In [20]:
#df.columns

In [49]:
sub_grade_columns = [col for col in df.columns if col.startswith('sub_grade')]
purpose_columns = [col for col in df.columns if col.startswith('purpose')]
empl_length_columns = [col for col in df.columns if col.startswith('emp_length')]

In [50]:
df.drop(columns=['Unnamed: 0','installment','total_acc'],inplace = True)

In [51]:
column_lists = [
sub_grade_columns,purpose_columns,empl_length_columns
]

In [52]:
for column_prefixes in column_lists:
    columns_to_drop = [col for col in df.columns if any(col.startswith(prefix) for prefix in column_prefixes)]
    df.drop(columns=columns_to_drop, inplace=True)

In [53]:
df.columns

Index(['loan_amnt', 'int_rate', 'annual_inc', 'dti', 'open_acc', 'pub_rec',
       'revol_bal', 'revol_util', 'mort_acc', 'pub_rec_bankruptcies',
       'term_ 60 months', 'grade_B', 'grade_C', 'grade_D', 'grade_E',
       'grade_F', 'grade_G', 'home_ownership_MORTGAGE', 'home_ownership_NONE',
       'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT',
       'verification_status_Source Verified', 'verification_status_Verified',
       'loan_status_Fully Paid', 'initial_list_status_w',
       'application_type_INDIVIDUAL', 'application_type_JOINT',
       'zip_column_05113', 'zip_column_11650', 'zip_column_22690',
       'zip_column_29597', 'zip_column_30723', 'zip_column_48052',
       'zip_column_70466', 'zip_column_86630', 'zip_column_93700'],
      dtype='object')

In [54]:
df.drop(columns=['pub_rec','mort_acc','initial_list_status_w'],inplace = True)

In [55]:
y = df['loan_status_Fully Paid']

In [56]:
df.drop(columns=['loan_status_Fully Paid'],inplace = True)

In [57]:
X = df.copy()

In [58]:
X

,loan_amnt,int_rate,annual_inc,dti,open_acc,revol_bal,revol_util,pub_rec_bankruptcies,term_ 60 months,grade_B,...,application_type_JOINT,zip_column_05113,zip_column_11650,zip_column_22690,zip_column_29597,zip_column_30723,zip_column_48052,zip_column_70466,zip_column_86630,zip_column_93700
0,6400.0,15.61,60000.0,25.72,12.0,31228.0,97.6,0.0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,25000.0,19.99,67000.0,20.49,18.0,24755.0,42.1,1.0,1,0,...,0,1,0,0,0,0,0,0,0,0
2,15000.0,5.32,59000.0,11.50,12.0,8925.0,23.3,0.0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,16000.0,15.61,72000.0,29.40,8.0,6972.0,79.2,0.0,0,0,...,0,1,0,0,0,0,0,0,0,0
4,8725.0,12.12,50000.0,22.88,10.0,25397.0,45.8,0.0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271512,10000.0,11.99,120000.0,9.61,15.0,7297.0,41.5,0.0,0,1,...,0,0,0,0,0,0,0,0,0,0
271513,26400.0,8.19,105000.0,17.04,5.0,14275.0,58.0,0.0,0,0,...,0,0,0,0,1,0,0,0,0,0
271514,4800.0,12.69,68000.0,14.59,6.0,1471.0,35.0,0.0,0,0,...,0,0,0,1,0,0,0,0,0,0
271515,20000.0,13.99,75000.0,24.56,23.0,13370.0,50.6,0.0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [59]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,shuffle = True,stratify = y)

In [60]:
scaler = MinMaxScaler()

# Fit and transform the scaler on training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data using the scaler fitted on training data
X_test_scaled = scaler.transform(X_test)

# Creating DataFrames from the scaled data
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

print("Original Train Data:")
print(X_train)
print("\nScaled Train Data using MinMaxScaler:")
print(X_train_scaled_df)

Original Train Data:
        loan_amnt  int_rate  annual_inc    dti  open_acc  revol_bal  \
238245    20950.0     15.61     48000.0  16.75      14.0    18509.0   
56518     15000.0     10.99     61000.0  28.66      18.0    24324.0   
160812    15000.0     13.05     55000.0  11.03       6.0    10795.0   
133353    17475.0     15.31     45000.0  24.03      20.0    26540.0   
261146     6400.0      9.99    154500.0  12.44       8.0     8126.0   
...           ...       ...         ...    ...       ...        ...   
97175     16800.0     11.99     94000.0   8.96      18.0    26823.0   
121073    10000.0     13.99     50000.0   7.87       4.0    12908.0   
76976     11000.0     10.99     56635.0   9.03       8.0     7794.0   
42687     12000.0     11.14     90000.0  14.64       8.0     6906.0   
201948    12000.0      6.49     60000.0  19.94      16.0     6197.0   

        revol_util  pub_rec_bankruptcies  term_ 60 months  grade_B  ...  \
238245        55.3                   0.0           

In [61]:
def training_predict(trainer, X_train, Y_train, X_test, Y_test):
    scores = {} # Initializing a Dictionary to store evaluation metrics
    
    trainer.fit(X_train, Y_train)  # Training
    Y_pred = trainer.predict(X_test) # Predicts
    
    '''
    Stores the result of trainer in its individual Key
    '''
    scores['auc'] = roc_auc_score(Y_test , trainer.predict_proba(X_test)[:,1])
    scores["AP-SCORE"]=average_precision_score(Y_test , trainer.predict_proba(X_test)[:,1])
    scores['Recall_test'] = recall_score(Y_test , Y_pred)
    scores['Precision_test'] = precision_score(Y_test , Y_pred)
    scores['F1_Score_test'] = f1_score(Y_test , Y_pred)
    scores['accuracy'] = accuracy_score(Y_test, Y_pred)
    
    return scores

In [62]:
y.value_counts()

0    217721
1     53796
Name: loan_status_Fully Paid, dtype: int64

In [66]:
xgb = XGBClassifier(random_state= 42,scale_pos_weight = 4)
rf = RandomForestClassifier(random_state= 42,class_weight='balanced')
gbm=GradientBoostingClassifier(random_state=42)
ada = AdaBoostClassifier(random_state = 42)
cat = CatBoostClassifier(class_weights=[0.3, 0.7], iterations=500, learning_rate=0.1, depth=16, random_state=1)
predictors = {} # Initializing a dictionary to store training and testing data
 
for model in [cat,xgb,rf,gbm]:
    model_name = model.__class__.__name__
    predictors[model_name]  = {}
    pipe=make_pipeline(model)
    predictors[model_name] =  training_predict(pipe, X_train_scaled, y_train, X_test_scaled, y_test) 
    #Scores that is returned from trainer goes to
    print(predictors)

0:	learn: 0.5657986	total: 2.69s	remaining: 22m 19s
1:	learn: 0.4908915	total: 2.82s	remaining: 11m 43s
2:	learn: 0.4539548	total: 5.57s	remaining: 15m 22s
3:	learn: 0.4273288	total: 8.26s	remaining: 17m 3s
4:	learn: 0.3978357	total: 11.1s	remaining: 18m 22s
5:	learn: 0.3869567	total: 14s	remaining: 19m 11s
6:	learn: 0.3787809	total: 16.9s	remaining: 19m 52s
7:	learn: 0.3718957	total: 19.8s	remaining: 20m 16s
8:	learn: 0.3661612	total: 22.6s	remaining: 20m 34s
9:	learn: 0.3615566	total: 25.4s	remaining: 20m 43s
10:	learn: 0.3576316	total: 28.2s	remaining: 20m 52s
11:	learn: 0.3544879	total: 31.1s	remaining: 21m 3s
12:	learn: 0.3513255	total: 34s	remaining: 21m 13s
13:	learn: 0.3472217	total: 36.8s	remaining: 21m 18s
14:	learn: 0.3419286	total: 39.7s	remaining: 21m 22s
15:	learn: 0.3369645	total: 42.6s	remaining: 21m 27s
16:	learn: 0.3340349	total: 45.4s	remaining: 21m 30s
17:	learn: 0.3301762	total: 48.3s	remaining: 21m 32s
18:	learn: 0.3270981	total: 51.1s	remaining: 21m 33s
19:	learn

153:	learn: 0.1509857	total: 7m 10s	remaining: 16m 6s
154:	learn: 0.1504180	total: 7m 13s	remaining: 16m 4s
155:	learn: 0.1493361	total: 7m 16s	remaining: 16m 2s
156:	learn: 0.1489634	total: 7m 19s	remaining: 16m
157:	learn: 0.1483941	total: 7m 22s	remaining: 15m 57s
158:	learn: 0.1477317	total: 7m 25s	remaining: 15m 54s
159:	learn: 0.1472622	total: 7m 28s	remaining: 15m 52s
160:	learn: 0.1460087	total: 7m 31s	remaining: 15m 50s
161:	learn: 0.1456169	total: 7m 34s	remaining: 15m 48s
162:	learn: 0.1450463	total: 7m 37s	remaining: 15m 45s
163:	learn: 0.1444398	total: 7m 40s	remaining: 15m 42s
164:	learn: 0.1439767	total: 7m 43s	remaining: 15m 40s
165:	learn: 0.1435181	total: 7m 46s	remaining: 15m 38s
166:	learn: 0.1428742	total: 7m 49s	remaining: 15m 36s
167:	learn: 0.1420956	total: 7m 52s	remaining: 15m 34s
168:	learn: 0.1412119	total: 7m 56s	remaining: 15m 32s
169:	learn: 0.1403431	total: 7m 59s	remaining: 15m 30s
170:	learn: 0.1397685	total: 8m 2s	remaining: 15m 28s
171:	learn: 0.1389

302:	learn: 0.0809212	total: 14m 51s	remaining: 9m 39s
303:	learn: 0.0805555	total: 14m 54s	remaining: 9m 36s
304:	learn: 0.0803960	total: 14m 57s	remaining: 9m 33s
305:	learn: 0.0800603	total: 15m 1s	remaining: 9m 31s
306:	learn: 0.0796998	total: 15m 4s	remaining: 9m 28s
307:	learn: 0.0795431	total: 15m 7s	remaining: 9m 25s
308:	learn: 0.0792201	total: 15m 11s	remaining: 9m 23s
309:	learn: 0.0787299	total: 15m 14s	remaining: 9m 20s
310:	learn: 0.0782138	total: 15m 17s	remaining: 9m 17s
311:	learn: 0.0777342	total: 15m 21s	remaining: 9m 15s
312:	learn: 0.0774520	total: 15m 24s	remaining: 9m 12s
313:	learn: 0.0770739	total: 15m 27s	remaining: 9m 9s
314:	learn: 0.0765626	total: 15m 31s	remaining: 9m 6s
315:	learn: 0.0762371	total: 15m 34s	remaining: 9m 4s
316:	learn: 0.0759553	total: 15m 37s	remaining: 9m 1s
317:	learn: 0.0758585	total: 15m 40s	remaining: 8m 58s
318:	learn: 0.0753488	total: 15m 43s	remaining: 8m 55s
319:	learn: 0.0751606	total: 15m 47s	remaining: 8m 52s
320:	learn: 0.074

453:	learn: 0.0456281	total: 22m 47s	remaining: 2m 18s
454:	learn: 0.0454242	total: 22m 50s	remaining: 2m 15s
455:	learn: 0.0452537	total: 22m 53s	remaining: 2m 12s
456:	learn: 0.0450452	total: 22m 56s	remaining: 2m 9s
457:	learn: 0.0448469	total: 22m 59s	remaining: 2m 6s
458:	learn: 0.0446488	total: 23m 2s	remaining: 2m 3s
459:	learn: 0.0445906	total: 23m 6s	remaining: 2m
460:	learn: 0.0445276	total: 23m 9s	remaining: 1m 57s
461:	learn: 0.0444616	total: 23m 12s	remaining: 1m 54s
462:	learn: 0.0444058	total: 23m 15s	remaining: 1m 51s
463:	learn: 0.0443156	total: 23m 18s	remaining: 1m 48s
464:	learn: 0.0441790	total: 23m 21s	remaining: 1m 45s
465:	learn: 0.0439731	total: 23m 25s	remaining: 1m 42s
466:	learn: 0.0438677	total: 23m 28s	remaining: 1m 39s
467:	learn: 0.0438358	total: 23m 31s	remaining: 1m 36s
468:	learn: 0.0436717	total: 23m 34s	remaining: 1m 33s
469:	learn: 0.0436273	total: 23m 38s	remaining: 1m 30s
470:	learn: 0.0435194	total: 23m 41s	remaining: 1m 27s
471:	learn: 0.043362

In [67]:
pd.DataFrame.from_dict(predictors).T

,auc,AP-SCORE,Recall_test,Precision_test,F1_Score_test,accuracy
CatBoostClassifier,0.889902,0.755690,0.542894,0.760844,0.633652,0.875626
XGBClassifier,0.906379,0.779895,0.790873,0.514668,0.623553,0.810806
RandomForestClassifier,0.890984,0.759260,0.457756,0.940065,0.615702,0.886786
GradientBoostingClassifier,0.899835,0.774411,0.436100,0.992596,0.605967,0.887633


Since we are prioritizing F1 score due to imbalanced nature(as well as both precision and recall are vital for a lending company) CatBoostClassifier is chosen as the given model which will be fine tuned using hyper-parameter tuning along with cross-validation

In [75]:
#Hyperparameter Tuning and Kfold Cross Validation

params = dict()
params['n_estimators'] = [100, 200, 300,400,500]
params['max_depth'] = [2,3,4,6,8,10,12]

cbc = CatBoostClassifier(**params,thread_count=8,task_type='GPU',scale_pos_weight = 4,verbose=True)

In [76]:
gscv = GridSearchCV(estimator = cbc, param_grid = params, scoring ='f1', cv = 5)

In [77]:
gscv.fit(X_train_scaled_df,y_train)

Learning rate set to 0.199791
0:	learn: 0.6089645	total: 8.38ms	remaining: 829ms
1:	learn: 0.5524978	total: 16.4ms	remaining: 806ms
2:	learn: 0.4816537	total: 23.5ms	remaining: 761ms
3:	learn: 0.4730554	total: 30.1ms	remaining: 723ms
4:	learn: 0.4603507	total: 39.1ms	remaining: 743ms
5:	learn: 0.4489191	total: 44.9ms	remaining: 703ms
6:	learn: 0.4384027	total: 56.8ms	remaining: 755ms
7:	learn: 0.4352045	total: 67.1ms	remaining: 772ms
8:	learn: 0.4324980	total: 76.9ms	remaining: 778ms
9:	learn: 0.4250100	total: 88.5ms	remaining: 796ms
10:	learn: 0.4183195	total: 98.8ms	remaining: 799ms
11:	learn: 0.4165847	total: 109ms	remaining: 800ms
12:	learn: 0.4154007	total: 120ms	remaining: 806ms
13:	learn: 0.4143852	total: 131ms	remaining: 807ms
14:	learn: 0.4134955	total: 141ms	remaining: 799ms
15:	learn: 0.4126008	total: 149ms	remaining: 782ms
16:	learn: 0.4117563	total: 155ms	remaining: 758ms
17:	learn: 0.4109252	total: 166ms	remaining: 758ms
18:	learn: 0.4102447	total: 176ms	remaining: 751ms


82:	learn: 0.3774625	total: 776ms	remaining: 159ms
83:	learn: 0.3773561	total: 793ms	remaining: 151ms
84:	learn: 0.3772311	total: 807ms	remaining: 142ms
85:	learn: 0.3770736	total: 816ms	remaining: 133ms
86:	learn: 0.3756114	total: 821ms	remaining: 123ms
87:	learn: 0.3754390	total: 827ms	remaining: 113ms
88:	learn: 0.3752783	total: 833ms	remaining: 103ms
89:	learn: 0.3751371	total: 839ms	remaining: 93.3ms
90:	learn: 0.3749970	total: 848ms	remaining: 83.8ms
91:	learn: 0.3748404	total: 854ms	remaining: 74.2ms
92:	learn: 0.3733320	total: 859ms	remaining: 64.7ms
93:	learn: 0.3731900	total: 865ms	remaining: 55.2ms
94:	learn: 0.3729635	total: 870ms	remaining: 45.8ms
95:	learn: 0.3728172	total: 876ms	remaining: 36.5ms
96:	learn: 0.3726245	total: 882ms	remaining: 27.3ms
97:	learn: 0.3710074	total: 889ms	remaining: 18.1ms
98:	learn: 0.3708179	total: 896ms	remaining: 9.04ms
99:	learn: 0.3705980	total: 901ms	remaining: 0us
Learning rate set to 0.199791
0:	learn: 0.6092125	total: 15ms	remaining: 1

51:	learn: 0.3861169	total: 577ms	remaining: 532ms
52:	learn: 0.3857083	total: 584ms	remaining: 518ms
53:	learn: 0.3854090	total: 591ms	remaining: 503ms
54:	learn: 0.3850958	total: 598ms	remaining: 489ms
55:	learn: 0.3847640	total: 605ms	remaining: 475ms
56:	learn: 0.3844712	total: 611ms	remaining: 461ms
57:	learn: 0.3841960	total: 618ms	remaining: 448ms
58:	learn: 0.3838662	total: 625ms	remaining: 434ms
59:	learn: 0.3835748	total: 633ms	remaining: 422ms
60:	learn: 0.3833022	total: 639ms	remaining: 409ms
61:	learn: 0.3829824	total: 647ms	remaining: 396ms
62:	learn: 0.3827019	total: 654ms	remaining: 384ms
63:	learn: 0.3824081	total: 662ms	remaining: 372ms
64:	learn: 0.3821489	total: 669ms	remaining: 360ms
65:	learn: 0.3818849	total: 676ms	remaining: 348ms
66:	learn: 0.3816467	total: 694ms	remaining: 342ms
67:	learn: 0.3814084	total: 704ms	remaining: 331ms
68:	learn: 0.3810894	total: 711ms	remaining: 319ms
69:	learn: 0.3808709	total: 717ms	remaining: 307ms
70:	learn: 0.3806800	total: 724

41:	learn: 0.4098233	total: 374ms	remaining: 1.41s
42:	learn: 0.4093847	total: 383ms	remaining: 1.4s
43:	learn: 0.4090640	total: 389ms	remaining: 1.38s
44:	learn: 0.4086732	total: 395ms	remaining: 1.36s
45:	learn: 0.4083190	total: 401ms	remaining: 1.34s
46:	learn: 0.4079807	total: 406ms	remaining: 1.32s
47:	learn: 0.4076252	total: 412ms	remaining: 1.3s
48:	learn: 0.4073077	total: 449ms	remaining: 1.38s
49:	learn: 0.4069782	total: 485ms	remaining: 1.46s
50:	learn: 0.4066685	total: 501ms	remaining: 1.46s
51:	learn: 0.4063959	total: 516ms	remaining: 1.47s
52:	learn: 0.4061301	total: 526ms	remaining: 1.46s
53:	learn: 0.4057929	total: 531ms	remaining: 1.44s
54:	learn: 0.4055455	total: 537ms	remaining: 1.42s
55:	learn: 0.4052861	total: 564ms	remaining: 1.45s
56:	learn: 0.4050092	total: 584ms	remaining: 1.46s
57:	learn: 0.4047719	total: 590ms	remaining: 1.44s
58:	learn: 0.4045526	total: 602ms	remaining: 1.44s
59:	learn: 0.4042824	total: 618ms	remaining: 1.44s
60:	learn: 0.4022885	total: 624ms

Learning rate set to 0.107362
0:	learn: 0.6345071	total: 7ms	remaining: 1.39s
1:	learn: 0.5991771	total: 13.6ms	remaining: 1.34s
2:	learn: 0.5463185	total: 21.3ms	remaining: 1.4s
3:	learn: 0.5286657	total: 28.4ms	remaining: 1.39s
4:	learn: 0.5124849	total: 36.7ms	remaining: 1.43s
5:	learn: 0.4959700	total: 44.7ms	remaining: 1.44s
6:	learn: 0.4829942	total: 50.1ms	remaining: 1.38s
7:	learn: 0.4704047	total: 60.6ms	remaining: 1.45s
8:	learn: 0.4674196	total: 66.9ms	remaining: 1.42s
9:	learn: 0.4601914	total: 75.9ms	remaining: 1.44s
10:	learn: 0.4529810	total: 80.9ms	remaining: 1.39s
11:	learn: 0.4461337	total: 86.3ms	remaining: 1.35s
12:	learn: 0.4442188	total: 92.8ms	remaining: 1.33s
13:	learn: 0.4425499	total: 99.8ms	remaining: 1.33s
14:	learn: 0.4411691	total: 108ms	remaining: 1.33s
15:	learn: 0.4398073	total: 115ms	remaining: 1.32s
16:	learn: 0.4348400	total: 121ms	remaining: 1.3s
17:	learn: 0.4299599	total: 126ms	remaining: 1.27s
18:	learn: 0.4290351	total: 132ms	remaining: 1.26s
19

...

14:	learn: 0.3858407	total: 908ms	remaining: 29.4s
15:	learn: 0.3842469	total: 969ms	remaining: 29.3s
16:	learn: 0.3828301	total: 1.03s	remaining: 29.3s
17:	learn: 0.3813603	total: 1.09s	remaining: 29.3s
18:	learn: 0.3799909	total: 1.16s	remaining: 29.4s
19:	learn: 0.3788122	total: 1.22s	remaining: 29.3s
20:	learn: 0.3776699	total: 1.28s	remaining: 29.3s
21:	learn: 0.3765990	total: 1.34s	remaining: 29.2s
22:	learn: 0.3754887	total: 1.41s	remaining: 29.2s
23:	learn: 0.3737138	total: 1.47s	remaining: 29.1s
24:	learn: 0.3727607	total: 1.53s	remaining: 29s
25:	learn: 0.3718079	total: 1.59s	remaining: 29s
26:	learn: 0.3709619	total: 1.65s	remaining: 28.9s
27:	learn: 0.3702712	total: 1.71s	remaining: 28.9s
28:	learn: 0.3693501	total: 1.78s	remaining: 28.9s
29:	learn: 0.3683657	total: 1.84s	remaining: 28.9s
30:	learn: 0.3675257	total: 1.91s	remaining: 28.9s
31:	learn: 0.3667053	total: 1.97s	remaining: 28.8s
32:	learn: 0.3654771	total: 2.03s	remaining: 28.8s
33:	learn: 0.3646023	total: 2.1s	re

178:	learn: 0.2929461	total: 11.6s	remaining: 20.7s
179:	learn: 0.2927403	total: 11.6s	remaining: 20.7s
180:	learn: 0.2924892	total: 11.7s	remaining: 20.6s
181:	learn: 0.2921507	total: 11.8s	remaining: 20.5s
182:	learn: 0.2917968	total: 11.8s	remaining: 20.5s
183:	learn: 0.2915951	total: 11.9s	remaining: 20.4s
184:	learn: 0.2910819	total: 12s	remaining: 20.4s
185:	learn: 0.2904705	total: 12s	remaining: 20.3s
186:	learn: 0.2900622	total: 12.1s	remaining: 20.2s
187:	learn: 0.2898596	total: 12.2s	remaining: 20.2s
188:	learn: 0.2895489	total: 12.2s	remaining: 20.1s
189:	learn: 0.2890436	total: 12.3s	remaining: 20s
190:	learn: 0.2886283	total: 12.4s	remaining: 20s
191:	learn: 0.2880097	total: 12.4s	remaining: 19.9s
192:	learn: 0.2875226	total: 12.5s	remaining: 19.9s
193:	learn: 0.2870076	total: 12.6s	remaining: 19.8s
194:	learn: 0.2866798	total: 12.6s	remaining: 19.7s
195:	learn: 0.2863818	total: 12.7s	remaining: 19.7s
196:	learn: 0.2861472	total: 12.8s	remaining: 19.6s
197:	learn: 0.285883

339:	learn: 0.2433830	total: 22.2s	remaining: 10.4s
340:	learn: 0.2432159	total: 22.2s	remaining: 10.4s
341:	learn: 0.2428012	total: 22.3s	remaining: 10.3s
342:	learn: 0.2423264	total: 22.4s	remaining: 10.2s
343:	learn: 0.2420747	total: 22.4s	remaining: 10.2s
344:	learn: 0.2418750	total: 22.5s	remaining: 10.1s
345:	learn: 0.2417355	total: 22.6s	remaining: 10s
346:	learn: 0.2413752	total: 22.6s	remaining: 9.98s
347:	learn: 0.2408602	total: 22.7s	remaining: 9.91s
348:	learn: 0.2404433	total: 22.8s	remaining: 9.85s
349:	learn: 0.2400905	total: 22.8s	remaining: 9.79s
350:	learn: 0.2396652	total: 22.9s	remaining: 9.72s
351:	learn: 0.2394780	total: 23s	remaining: 9.65s
352:	learn: 0.2391570	total: 23s	remaining: 9.59s
353:	learn: 0.2387964	total: 23.1s	remaining: 9.53s
354:	learn: 0.2385027	total: 23.2s	remaining: 9.46s
355:	learn: 0.2381434	total: 23.2s	remaining: 9.4s
356:	learn: 0.2378188	total: 23.3s	remaining: 9.33s
357:	learn: 0.2375482	total: 23.4s	remaining: 9.27s
358:	learn: 0.23729

498:	learn: 0.2040916	total: 32.6s	remaining: 65.4ms
499:	learn: 0.2038997	total: 32.7s	remaining: 0us


GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostClassifier object at 0x00000215BD20B850>,
             param_grid={'max_depth': [2, 3, 4, 6, 8, 10, 12],
                         'n_estimators': [100, 200, 300, 400, 500]},
             scoring='f1')

In [79]:
#returns the estimator with the best performance
print(gscv.best_estimator_)

#returns the best score
print(gscv.best_score_)

#returns the best parameters
print(gscv.best_params_)

0.6412576663191725
{'max_depth': 12, 'n_estimators': 500}


We can observe only a marginal increase in the F1 score after performing a 10 fold Cross validation with hyperparameter tuning

In [87]:
gsvc2 = CatBoostClassifier(max_depth = 16,n_estimators= 500,thread_count=8,task_type='CPU',scale_pos_weight = 4,verbose=True,random_state = 1)

In [88]:
gsvc2.fit(X_train_scaled_df,y_train)

Learning rate set to 0.193566
0:	learn: 0.5334636	total: 2.89s	remaining: 24m
1:	learn: 0.4254654	total: 5.6s	remaining: 23m 15s
2:	learn: 0.4013001	total: 8.31s	remaining: 22m 57s
3:	learn: 0.3898505	total: 11s	remaining: 22m 48s
4:	learn: 0.3788420	total: 13.7s	remaining: 22m 40s
5:	learn: 0.3669318	total: 16.5s	remaining: 22m 38s
6:	learn: 0.3568780	total: 19.3s	remaining: 22m 36s
7:	learn: 0.3526697	total: 22s	remaining: 22m 34s
8:	learn: 0.3388277	total: 24.8s	remaining: 22m 34s
9:	learn: 0.3296305	total: 27.6s	remaining: 22m 32s
10:	learn: 0.3219885	total: 30.4s	remaining: 22m 31s
11:	learn: 0.3154223	total: 33.2s	remaining: 22m 30s
12:	learn: 0.3124521	total: 35.9s	remaining: 22m 26s
13:	learn: 0.3094550	total: 38.7s	remaining: 22m 23s
14:	learn: 0.3062829	total: 41.4s	remaining: 22m 18s
15:	learn: 0.3019954	total: 44.1s	remaining: 22m 14s
16:	learn: 0.2995461	total: 46.9s	remaining: 22m 11s
17:	learn: 0.2961458	total: 49.6s	remaining: 22m 8s
18:	learn: 0.2945417	total: 52.4s	re

152:	learn: 0.0790434	total: 7m 3s	remaining: 16m
153:	learn: 0.0782779	total: 7m 6s	remaining: 15m 58s
154:	learn: 0.0772002	total: 7m 9s	remaining: 15m 55s
155:	learn: 0.0762499	total: 7m 12s	remaining: 15m 52s
156:	learn: 0.0753522	total: 7m 14s	remaining: 15m 49s
157:	learn: 0.0746655	total: 7m 17s	remaining: 15m 47s
158:	learn: 0.0738457	total: 7m 20s	remaining: 15m 44s
159:	learn: 0.0735613	total: 7m 23s	remaining: 15m 42s
160:	learn: 0.0731403	total: 7m 26s	remaining: 15m 39s
161:	learn: 0.0725139	total: 7m 29s	remaining: 15m 37s
162:	learn: 0.0722984	total: 7m 31s	remaining: 15m 34s
163:	learn: 0.0716980	total: 7m 34s	remaining: 15m 31s
164:	learn: 0.0715413	total: 7m 37s	remaining: 15m 28s
165:	learn: 0.0709038	total: 7m 40s	remaining: 15m 26s
166:	learn: 0.0695997	total: 7m 43s	remaining: 15m 23s
167:	learn: 0.0689587	total: 7m 46s	remaining: 15m 21s
168:	learn: 0.0682685	total: 7m 48s	remaining: 15m 18s
169:	learn: 0.0677446	total: 7m 51s	remaining: 15m 15s
170:	learn: 0.067

301:	learn: 0.0288655	total: 14m 8s	remaining: 9m 16s
302:	learn: 0.0287211	total: 14m 11s	remaining: 9m 13s
303:	learn: 0.0285285	total: 14m 14s	remaining: 9m 10s
304:	learn: 0.0282669	total: 14m 17s	remaining: 9m 8s
305:	learn: 0.0282303	total: 14m 20s	remaining: 9m 5s
306:	learn: 0.0280730	total: 14m 22s	remaining: 9m 2s
307:	learn: 0.0278040	total: 14m 25s	remaining: 8m 59s
308:	learn: 0.0276250	total: 14m 28s	remaining: 8m 56s
309:	learn: 0.0274420	total: 14m 31s	remaining: 8m 54s
310:	learn: 0.0272056	total: 14m 34s	remaining: 8m 51s
311:	learn: 0.0270451	total: 14m 36s	remaining: 8m 48s
312:	learn: 0.0268099	total: 14m 39s	remaining: 8m 45s
313:	learn: 0.0267469	total: 14m 42s	remaining: 8m 42s
314:	learn: 0.0264917	total: 14m 45s	remaining: 8m 39s
315:	learn: 0.0263651	total: 14m 48s	remaining: 8m 37s
316:	learn: 0.0261945	total: 14m 50s	remaining: 8m 34s
317:	learn: 0.0259964	total: 14m 53s	remaining: 8m 31s
318:	learn: 0.0259289	total: 14m 56s	remaining: 8m 28s
319:	learn: 0.

452:	learn: 0.0150841	total: 21m 7s	remaining: 2m 11s
453:	learn: 0.0150605	total: 21m 10s	remaining: 2m 8s
454:	learn: 0.0149734	total: 21m 13s	remaining: 2m 5s
455:	learn: 0.0148981	total: 21m 15s	remaining: 2m 3s
456:	learn: 0.0148812	total: 21m 18s	remaining: 2m
457:	learn: 0.0148663	total: 21m 21s	remaining: 1m 57s
458:	learn: 0.0148281	total: 21m 24s	remaining: 1m 54s
459:	learn: 0.0147970	total: 21m 27s	remaining: 1m 51s
460:	learn: 0.0147630	total: 21m 29s	remaining: 1m 49s
461:	learn: 0.0147286	total: 21m 32s	remaining: 1m 46s
462:	learn: 0.0146778	total: 21m 35s	remaining: 1m 43s
463:	learn: 0.0146246	total: 21m 38s	remaining: 1m 40s
464:	learn: 0.0145984	total: 21m 41s	remaining: 1m 37s
465:	learn: 0.0145367	total: 21m 43s	remaining: 1m 35s
466:	learn: 0.0145066	total: 21m 46s	remaining: 1m 32s
467:	learn: 0.0144768	total: 21m 49s	remaining: 1m 29s
468:	learn: 0.0144327	total: 21m 52s	remaining: 1m 26s
469:	learn: 0.0143796	total: 21m 55s	remaining: 1m 23s
470:	learn: 0.0143

In [89]:
predictions = model.predict(X_test_scaled_df)

# Calculate F1 score
f1 = f1_score(y_test, predictions)

print("F1 Score:", f1)

F1 Score: 0.6059666795815576


C:\Users\Kumar\anaconda3\Lib\site-packages\sklearn\base.py:457: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(
